# Websocket Message를 종류별로 구성함.

## 타입별로 웹소켓 데이터를 수신하고 분석용도로 사용한다.
- ticker: 개별 심볼에 대한 전체 티커 정보 제공
- trade: 개별 거래 정보 제공
    - "m" : 매도자 관점 taker주문(시장가)
- miniTicker: 심볼별 간소화된 티커 정보 제공
- depth: 주문서 정보 제공
- aggTrade: 집계된 거래 정보 제공

# 웹소켓 데이터 저장 스토리지 클라스

In [1]:

import asyncio
import nest_asyncio
from pprint import pprint
from typing import List
import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))
from Workspace.Services.PublicData.Receiver.FuturesMarketWebsocket import FuturesMarketWebsocket

nest_asyncio.apply()
symbol = ["BTCUSDT"]
instance = FuturesMarketWebsocket(symbol)
ws_types = ["ticker", "trade", "miniTicker", "depth", "aggTrade"]

class storage:
    __slots__ = tuple(ws_types)
    def __init__(self):
        for attr in self.__slots__:
            setattr(self, attr, [])

    def set_data(self, ws_type:str, data:List):
        setattr(self, ws_type, data)
    
    def get_data(self, ws_type:str):
        return getattr(self, ws_type)

storage_message = storage()

In [2]:
async def main(stream_type):
    print(f"  ⏳ Websocket Stream({stream_type}) 연결 시도.")
    await instance.open_stream_connection(stream_type)
    print(f"  🔗 Websocket Stream({stream_type}) 연결 완료.")
    message = await instance.receive_message()
    print(f"  📩 message 수신")
    pprint(message)
    await instance.close_connection()
    print(f"  ⛓️‍💥 Websocket Stream({stream_type}) 연결 종료.")

async def ws_cycle(stream_type:str, count:int):
    print(f"  ⏳ Websocket Stream({stream_type}) 연결 시도.")
    await instance.open_stream_connection(stream_type)
    print(f"  🔗 Websocket Stream({stream_type}) 연결 완료.")
    print(f"  📩 message 수신 시작")
    result = []
    for _ in range(count):
        message = await instance.receive_message()
        result.append(message)
    print(f"  📩 message 수신 완료")
    await instance.close_connection()
    print(f"  ⛓️‍💥 Websocket Stream({stream_type}) 연결 종료.")
    return result

In [3]:
ws_type = ws_types[0]
receiver_count = 10
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(ticker) 연결 시도.
  🔗 Websocket Stream(ticker) 연결 완료.
  📩 message 수신 시작
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(ticker) 연결 종료.
[{'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742035404569, 's': 'BTCUSDT', 'p': '759.20', 'P': '0.915', 'w': '83964.93', 'c': '83770.10', 'Q': '0.002', 'o': '83010.90', 'h': '85270.20', 'l': '82594.30', 'v': '208452.292', 'q': '17502681768.23', 'O': 1741948980000, 'C': 1742035404566, 'F': 6103511267, 'L': 6106812234, 'n': 3300111}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742035406348, 's': 'BTCUSDT', 'p': '759.10', 'P': '0.914', 'w': '83964.93', 'c': '83770.00', 'Q': '0.003', 'o': '83010.90', 'h': '85270.20', 'l': '82594.30', 'v': '208452.801', 'q': '17502724407.18', 'O': 1741948980000, 'C': 1742035406345, 'F': 6103511267, 'L': 6106812259, 'n': 3300136}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742035407305, 's': 'BTCUSDT', 'p': '759.10', 'P': '0.914', 'w': '83964.93', 'c': '8

In [4]:
ws_type = ws_types[1]
receiver_count = 2000
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(trade) 연결 시도.
  🔗 Websocket Stream(trade) 연결 완료.
  📩 message 수신 시작
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(trade) 연결 종료.
[{'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742035423027, 'T': 1742035423027, 's': 'BTCUSDT', 't': 6106812574, 'p': '83770.10', 'q': '0.023', 'X': 'MARKET', 'm': False}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742035423029, 'T': 1742035423028, 's': 'BTCUSDT', 't': 6106812575, 'p': '83770.00', 'q': '0.006', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742035423046, 'T': 1742035423046, 's': 'BTCUSDT', 't': 6106812576, 'p': '83770.00', 'q': '0.012', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742035423049, 'T': 1742035423049, 's': 'BTCUSDT', 't': 6106812577, 'p': '83770.00', 'q': '0.006', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742035423188, 'T': 1742035423188, 's': 'BTCUSDT', 't': 6106812578

In [5]:
ws_type = ws_types[2]
receiver_count = 10
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(miniTicker) 연결 시도.
  🔗 Websocket Stream(miniTicker) 연결 완료.
  📩 message 수신 시작
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(miniTicker) 연결 종료.
[{'stream': 'btcusdt@miniTicker', 'data': {'e': '24hrMiniTicker', 'E': 1742035562803, 's': 'BTCUSDT', 'c': '83793.10', 'o': '83011.40', 'h': '85270.20', 'l': '82594.30', 'v': '208078.644', 'q': '17471739549.14'}}, {'stream': 'btcusdt@miniTicker', 'data': {'e': '24hrMiniTicker', 'E': 1742035564240, 's': 'BTCUSDT', 'c': '83793.10', 'o': '83011.40', 'h': '85270.20', 'l': '82594.30', 'v': '208079.158', 'q': '17471782618.79'}}, {'stream': 'btcusdt@miniTicker', 'data': {'e': '24hrMiniTicker', 'E': 1742035567054, 's': 'BTCUSDT', 'c': '83790.20', 'o': '83011.40', 'h': '85270.20', 'l': '82594.30', 'v': '208080.298', 'q': '17471878142.60'}}, {'stream': 'btcusdt@miniTicker', 'data': {'e': '24hrMiniTicker', 'E': 1742035568674, 's': 'BTCUSDT', 'c': '83790.30', 'o': '83011.40', 'h': '85270.20', 'l': '82594.30', 'v': '208080.787', 'q': '1747191

In [6]:
ws_type = ws_types[3]
receiver_count = 10
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(depth) 연결 시도.
  🔗 Websocket Stream(depth) 연결 완료.
  📩 message 수신 시작
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(depth) 연결 종료.
[{'stream': 'btcusdt@depth', 'data': {'e': 'depthUpdate', 'E': 1742035581749, 'T': 1742035581746, 's': 'BTCUSDT', 'U': 7032044089801, 'u': 7032044103193, 'pu': 7032044089528, 'b': [['1000.00', '371.067'], ['5000.00', '5.598'], ['33000.00', '9.960'], ['34000.00', '3.058'], ['50000.00', '14.095'], ['67032.20', '0.000'], ['75411.20', '0.003'], ['79600.00', '13.441'], ['81234.60', '0.000'], ['82114.40', '1.957'], ['82540.90', '0.583'], ['82613.30', '0.100'], ['82952.30', '0.000'], ['82984.90', '0.364'], ['83091.80', '4.002'], ['83371.20', '0.014'], ['83432.70', '0.000'], ['83455.00', '0.081'], ['83566.10', '0.000'], ['83566.20', '0.274'], ['83647.80', '0.320'], ['83648.30', '0.002'], ['83650.00', '5.087'], ['83653.60', '0.000'], ['83655.50', '0.000'], ['83673.50', '0.193'], ['83677.20', '0.119'], ['83679.20', '0.085'], ['83682.30', '0.006'], ['8369

In [7]:
ws_type = ws_types[4]
receiver_count = 10
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(aggTrade) 연결 시도.
  🔗 Websocket Stream(aggTrade) 연결 완료.
  📩 message 수신 시작
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(aggTrade) 연결 종료.
[{'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 1742035584728, 'a': 2634996904, 's': 'BTCUSDT', 'p': '83790.30', 'q': '0.007', 'f': 6106814715, 'l': 6106814715, 'T': 1742035584573, 'm': False}}, {'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 1742035585223, 'a': 2634996905, 's': 'BTCUSDT', 'p': '83790.20', 'q': '0.005', 'f': 6106814716, 'l': 6106814716, 'T': 1742035585069, 'm': True}}, {'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 1742035585618, 'a': 2634996906, 's': 'BTCUSDT', 'p': '83790.20', 'q': '0.020', 'f': 6106814717, 'l': 6106814717, 'T': 1742035585463, 'm': True}}, {'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 1742035586010, 'a': 2634996907, 's': 'BTCUSDT', 'p': '83790.20', 'q': '0.100', 'f': 6106814718, 'l': 6106814722, 'T': 1742035585856, 'm': True}}, {'stream': 'b